In [140]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [141]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5
no_cuda = False
log_interval = 100

# Model

In [142]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [143]:
train_paths = glob('/Users/eunchankim/Desktop/dataset/cifar/train/*.png')
test_paths = glob('/Users/eunchankim/Desktop/dataset/cifar/test/*.png')

len(train_paths), len(test_paths)

(50000, 10000)

In [144]:
train_paths[0]

'/Users/eunchankim/Desktop/dataset/cifar/train/32270_deer.png'

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [145]:
# path = train_paths[0]
# path

# lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
# lbl_name

def get_label(path):
    lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
    label = np.argmax(classes == lbl_name)
    return label

# img_pil = Image.open(path)
# image = np.array(img_pil)
# image.shape

In [146]:
class Dataset(Dataset):
    
    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform
    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        # read image
        image = Image.open(path)
        
        # get label
        label = get_label(path)
        if self.transform:
            image = self.transform(image)

        return image, label

In [147]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else "cpu")

kwargs ={'num_workers':1, 'pin_menory':True} if use_cuda else {}

In [148]:
# train
train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths,
           transforms.Compose([
               transforms.RandomHorizontalFlip(),
               transforms.ToTensor(),
               transforms.Normalize(
               mean=[0.406],
               std=[0.225]
               )])),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
    )

# test
test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize(
               mean=[0.406],
               std=[0.225]
               )])),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
    )

# Optimization

In [149]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Training

In [150]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.311073
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.279047
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.279692
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.248241
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.241749
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.188693
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.081832
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.074457

Test set: Average loss: 2.0560, Accuracy: 2695/10000 (27%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 2.159764
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.963291
Train Epoch: 2 [12800/50000 (26%)]	Loss: 2.089917
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.862567
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.991901
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.977339
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.976192
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.935786

Test set: Average loss: 1.9117, Accuracy: 3265/10000 (33%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 2.015244
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.805